In [13]:
%run ./Component.ipynb
%run ./Game.ipynb
%run ./Score.ipynb

In [20]:
import time
import json
from selenium import webdriver
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC 

class TicTacToeApp:
    def __init__(self):
        self.wd = webdriver.Firefox(executable_path=r'C:\Projects\TicTacVue\geckodriver.exe')
        #self.wd = webdriver.Chrome('C:/Projects/TicTacVue/chromedriver.exe')
        #self.wd.get('http://jerichokain.github.io/TicTacVue')
        self.wd.get('http://127.0.0.1:8081/')
        time.sleep(2) #gets cache and dom renders
        self.components = dict([(component.__name__, component) for component in Component.__subclasses__()])
    
    def __del__(self):
        self.wd.close()
    
#     def _get_shadow_root(self, elt):
#         return self.wd.execute_script('return arguments[0].shadowRoot', elt)

    def get_component(self, test, page_component):
        return self.components[page_component](self, test)
        
    def _get_component(self, page_component):
        elt = self.wd.find_element_by_id('app')
        return elt.find_element_by_css_selector(page_component)
        #this is for shadow roots / webcomponents in polymer
        #elt2 = self._get_shadow_root(elt)
        #elt3 = elt2.find_element_by_css_selector(page_component)
        #return self._get_shadow_root(elt3)
                      
    def _css_has_text(self, web_element, css, text):
        elts = filter(lambda e: 'display: none' not in [s.strip(' ') for s in e.get_attribute('style').split(';')], 
                      web_element.find_elements_by_css_selector(css))
        texts = [e.text for e in elts]
        if text in texts:
            return True
        raise AssertionError(f'No {css} with text="{text}" found. Available matches are [{", ".join(texts)}].')
    
#app = TicTacToeApp()

Game
